# Análise Inicial de Transações - Projeto AML/Fraud
Este notebook realiza a exploração inicial do dataset de transações simuladas.

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar os dados
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregar os dados direto do GitHub
url = "https://raw.githubusercontent.com/a-bucci/AML---Fraud---AI---Portfolio/main/1_fiat_analysis/aml_fraud_risk_simulated_v1.csv"
df = pd.read_csv(url)

# Visualizar as 5 primeiras linhas
df.head()

,transaction_id,amount,account_age_days,transaction_hour,customer_risk_score,origin_country,destination_country,device_type,past_alerts
0,T0000,938.54,531,6,69,RU,RU,mobile,3
1,T0001,6020.24,988,16,82,FR,MX,desktop,0
2,T0002,2633.49,1198,19,29,BR,RU,desktop,2
3,T0003,1825.89,1254,8,70,NG,MX,mobile,2
4,T0004,339.25,958,16,73,NG,US,mobile,1


## Informações Gerais
Aqui analisamos dimensões, tipos de dados e valores ausentes.

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

## Distribuição de Variáveis
Visualizações para entender a distribuição de valores.

In [ ]:
plt.figure(figsize=(10, 6))
df['amount'].hist(bins=50)
plt.title('Distribuição de Valores de Transações')
plt.xlabel('Valor (USD)')
plt.ylabel('Frequência')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='origin_country', order=df['origin_country'].value_counts().index)
plt.title('Origem das Transações')
plt.ylabel('Contagem')
plt.grid(True)
plt.show()

## Correlação entre variáveis numéricas

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()

### Próximos passos:
- Criar score manual de risco com base em regras simples
- Gerar coluna `manual_risk_score`
- Analisar comportamento de alto risco